In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 2 - M. Massot 2022-2023 - Ecole polytechnique
#    ----------   
#    Conditionnement et stabilité : fonction de Runge
#    
#    Auteurs : L. Séries et M. Massot - (C) 2022
#    

# Conditionnement et stabilité : fonction de Runge

In [ ]:
import numpy as np
from mpmath import mp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "seaborn"
marker=dict(size=5, symbol='x-thin', line=dict(width=1, color='rgb(76,114,176)'))
marker2=dict(size=5, symbol='x-thin', line=dict(width=1, color='rgb(221,132,82)'))

def compute_divided_diff_coef(x, y):
    n = x.size
    coef = y.copy()
    for i in range(1,n):
        coef[i:] = (coef[i:] - coef[i-1:-1])/(x[i:] - x[:-i])
    return coef

def newton_interp(xi, yi, x):
    coef = compute_divided_diff_coef(xi, yi)
    n = xi.size
    p = np.zeros(x.size)
    for i in range(n-1,0,-1):
        p = (coef[i]+p) * (x-xi[i-1])
    p = p +  coef[0]
    return p

def cheb_points(a, b, n):
    x = np.zeros(n+1)
    for i in range(n+1):
        x[i] = (a+b)/2 + ((b-a)/2) * np.cos(((2*i+1)*np.pi)/(2*n + 2))
    return x

def lagrange(k, xk, x):
    lag = np.ones(x.size)
    for j in range(xk.size):
        if (j!=k): lag *= (x-xk[j])/(xk[k]-xk[j])
    return lag

def compute_bary_coef(xk):
    n = xk.size
    wk = 0. * np.copy(xk)

    for k in range(n):
        wk[k] = 1. / (np.prod( xk[k] - xk[np.arange(n)!=k] ))
        
    return wk

def improved_lag_interp(xk, yk, x):
    n = xk.size
    l = 0. * np.copy(x) + 1.
    p = 0. * np.copy(x)
    
    wk = compute_bary_coef(xk)
    
    mask = np.in1d(x, xk[:])
    p[np.where(mask)] = yk[np.where(np.in1d(xk, x))]
    mask = np.invert(mask)    

    for k in range(n):
        l[mask] *= x[mask] - xk[k]
        p[mask] += (wk[k]/(x[mask] - xk[k])) * yk[k]
        
    return l*p

def compute_cond(xk, yk, pn, x):
    cond = np.zeros(x.size)
    for k in range(xk.size):
        cond += np.abs(lagrange(k, xk, x) * yk[k]) / np.abs(pn)
    return cond 

def interp(xk, yk, method="newton"):
    if method == "newton":
        return lambda x: newton_interp(xk, yk, x)
    elif method == "improved_lag":
        return lambda x: improved_lag_interp(xk, yk, x)
    else:
        print(f"method {method} is not implemented\n")
        error(1)

On considère l'interpolation de la fonction $f(x) = \displaystyle \frac{1}{(1 + 25x^2)}$.

In [ ]:
def f(x):
    return 1/(1+25*x*x)

## Méthode de Newton

In [ ]:
n = 60
xmin = -1.
xmax = 1.

# point d'interpolation
xk_equi = np.linspace(xmin, xmax, n+1)
yk_equi = f(xk_equi)
xk_cheb = cheb_points(xmin, xmax, n+1)
yk_cheb = f(xk_cheb)

# points auxquels le polynome d'interpolation sera évalués pour le tracé
xplot = np.linspace(xmin, xmax, 500)

# polynome calculé par la méthode de Newton
pn_equi = newton_interp(xk_equi, yk_equi, xplot)
pn_cheb = newton_interp(xk_cheb, yk_cheb, xplot)

# calcul de reference à précision fixée
mp.prec = 113
xk_equi_ref = mp.mpf('1') * xk_equi 
yk_equi_ref = f(xk_equi_ref)
xk_cheb_ref = mp.mpf('1') * xk_cheb 
yk_cheb_ref =  f(xk_cheb_ref)
xplot_ref = mp.mpf('1') * xplot 
pn_equi_ref = newton_interp(xk_equi_ref, yk_equi_ref, xplot_ref)
pn_equi_ref = np.array(pn_equi_ref, dtype=float)
pn_cheb_ref = newton_interp(xk_cheb_ref, yk_cheb_ref, xplot_ref)
pn_cheb_ref = np.array(pn_cheb_ref, dtype=float)

# points auxquels le calcul de l'erreur et le conditionnement relatif local sont calculés
x_equi_err = np.linspace(xmin, xmax, 100)
# on retire les points xk de x_err
x_equi_err = np.setdiff1d(x_equi_err, xk_equi)
x_equi_err_ref = mp.mpf('1') * x_equi_err
p_equi_err = newton_interp(xk_equi, yk_equi, x_equi_err)
p_equi_err_ref = newton_interp(xk_equi_ref, yk_equi_ref, x_equi_err_ref)
err_equi = np.abs((p_equi_err - p_equi_err_ref)/p_equi_err_ref)
err_equi = np.array(err_equi, dtype=float)

x_cheb_err = np.linspace(xmin, xmax, 100)
# on retire les points xk de x_err
x_cheb_err = np.setdiff1d(x_cheb_err, xk_cheb)
x_cheb_err_ref = mp.mpf('1') * x_cheb_err
p_cheb_err = newton_interp(xk_cheb, yk_cheb, x_cheb_err)
p_cheb_err_ref = newton_interp(xk_cheb_ref, yk_cheb_ref, x_cheb_err_ref)
err_cheb = np.abs((p_cheb_err - p_cheb_err_ref)/p_cheb_err_ref)
err_cheb = np.array(err_cheb, dtype=float)
    
# conditionnement
cond_equi = compute_cond(xk_equi, yk_equi, np.array(p_equi_err_ref, dtype=float), x_equi_err)
cond_cheb = compute_cond(xk_cheb, yk_cheb, np.array(p_cheb_err_ref, dtype=float), x_cheb_err)
    
# erreur / conditionnement 
dd_equi = err_equi/cond_equi
dd_cheb = err_cheb/cond_cheb

fig = make_subplots(rows=4, cols=1, vertical_spacing=0.1,
                    subplot_titles=("Approximation polynomiale", "Erreur relative", 
                                    "Conditionnement relatif local", "Erreur relative / Conditionnement"))

fig.add_trace(go.Scatter(x=xplot, y=f(xplot), name="f(x)", legendgroup='1'), row=1, col=1)
fig.add_trace(go.Scatter(x=xplot, y=pn_cheb, name=f"pn(x)", legendgroup='1'), row=1, col=1)
fig.add_trace(go.Scatter(x=xplot, y=pn_cheb_ref, name=f"pn(x) ref", legendgroup='1'), row=1, col=1)
fig.add_trace(go.Scatter(x=xk_cheb, y=yk_cheb, mode='markers', name="pts d'interpolation", legendgroup='1'), row=1, col=1)
fig.update_yaxes(range=[-0.5, 1.1], row=1)

fig.add_trace(go.Scatter(x=x_cheb_err, y=err_cheb, name="err", mode='markers', marker=marker, legendgroup='2', showlegend=False), row=2, col=1)
fig.update_yaxes(type="log", exponentformat='e', row=2)

fig.add_trace(go.Scatter(x=x_cheb_err, y=cond_cheb, name="cond", mode='markers', marker=marker, legendgroup='3', showlegend=False), row=3, col=1)
fig.update_yaxes(type="log", exponentformat='e', row=3)

fig.add_trace(go.Scatter(x=x_cheb_err, y=dd_cheb, name="erreur rel / conditionnement", mode='markers', marker=marker, legendgroup='4', showlegend=False), row=4, col=1)
fig.update_yaxes(type="log", exponentformat='e', row=4, col=1)


buttons=[dict(label="cheb.", method="update", args=[{"x":[xplot, xplot, xplot, xk_cheb, x_cheb_err, x_cheb_err, x_cheb_err], 
                                                     "y":[f(xplot), pn_cheb, pn_cheb_ref, yk_cheb, err_cheb, cond_cheb, dd_cheb]}]),
         dict(label="equi.", method="update", args=[{"x":[xplot, xplot, xplot, xk_equi, x_equi_err, x_equi_err, x_equi_err], 
                                                     "y":[f(xplot), pn_equi, pn_equi_ref, yk_equi, err_equi, cond_equi, dd_equi]}])]

fig.update_layout(updatemenus=[dict(type="buttons", direction="right", buttons=buttons, x=0., xanchor="left", y=1.1, yanchor="top")], 
                  height=1500, legend_tracegroupgap = 320, legend_groupclick="toggleitem")

## Première forme de la formule d’interpolation barycentrique

In [ ]:
n = 60
xmin = -1.
xmax = 1.

# point d'interpolation
xk_equi = np.linspace(xmin, xmax, n+1)
yk_equi = f(xk_equi)
xk_cheb = cheb_points(xmin, xmax, n+1)
yk_cheb = f(xk_cheb)

# points auxquels le polynome d'interpolation sera évalués pour le tracé
xplot = np.linspace(xmin, xmax, 500)

# polynome calculé par la méthode de Newton
pn_equi = improved_lag_interp(xk_equi, yk_equi, xplot)
pn_cheb = improved_lag_interp(xk_cheb, yk_cheb, xplot)

# calcul de reference à précision fixée
mp.prec = 113
xk_equi_ref = mp.mpf('1') * xk_equi 
yk_equi_ref = f(xk_equi_ref)
xk_cheb_ref = mp.mpf('1') * xk_cheb 
yk_cheb_ref =  f(xk_cheb_ref)
xplot_ref = mp.mpf('1') * xplot 
pn_equi_ref = improved_lag_interp(xk_equi_ref, yk_equi_ref, xplot_ref)
pn_equi_ref = np.array(pn_equi_ref, dtype=float)
pn_cheb_ref = improved_lag_interp(xk_cheb_ref, yk_cheb_ref, xplot_ref)
pn_cheb_ref = np.array(pn_cheb_ref, dtype=float)

# points auxquels le calcul de l'erreur et le conditionnement relatif local sont calculés
x_equi_err = np.linspace(xmin, xmax, 100)
# on retire les points xk de x_err
x_equi_err = np.setdiff1d(x_equi_err, xk_equi)
x_equi_err_ref = mp.mpf('1') * x_equi_err
p_equi_err = improved_lag_interp(xk_equi, yk_equi, x_equi_err)
p_equi_err_ref = improved_lag_interp(xk_equi_ref, yk_equi_ref, x_equi_err_ref)
err_equi = np.abs((p_equi_err - p_equi_err_ref)/p_equi_err_ref)
err_equi = np.array(err_equi, dtype=float)

x_cheb_err = np.linspace(xmin, xmax, 100)
# on retire les points xk de x_err
x_cheb_err = np.setdiff1d(x_cheb_err, xk_cheb)
x_cheb_err_ref = mp.mpf('1') * x_cheb_err
p_cheb_err = improved_lag_interp(xk_cheb, yk_cheb, x_cheb_err)
p_cheb_err_ref = improved_lag_interp(xk_cheb_ref, yk_cheb_ref, x_cheb_err_ref)
err_cheb = np.abs((p_cheb_err - p_cheb_err_ref)/p_cheb_err_ref)
err_cheb = np.array(err_cheb, dtype=float)
    
# conditionnement
cond_equi = compute_cond(xk_equi, yk_equi, np.array(p_equi_err_ref, dtype=float), x_equi_err)
cond_cheb = compute_cond(xk_cheb, yk_cheb, np.array(p_cheb_err_ref, dtype=float), x_cheb_err)
    
# erreur / conditionnement 
dd_equi = err_equi/cond_equi
dd_cheb = err_cheb/cond_cheb

fig = make_subplots(rows=4, cols=1, vertical_spacing=0.1,
                    subplot_titles=("Approximation polynomiale", "Erreur relative", 
                                    "Conditionnement relatif local", "Erreur relative / Conditionnement"))

fig.add_trace(go.Scatter(x=xplot, y=f(xplot), name="f(x)", legendgroup='1'), row=1, col=1)
fig.add_trace(go.Scatter(x=xplot, y=pn_cheb, name=f"pn(x)", legendgroup='1'), row=1, col=1)
fig.add_trace(go.Scatter(x=xplot, y=pn_cheb_ref, name=f"pn(x) ref", legendgroup='1'), row=1, col=1)
fig.add_trace(go.Scatter(x=xk_cheb, y=yk_cheb, mode='markers', name="pts d'interpolation", legendgroup='1'), row=1, col=1)
fig.update_yaxes(range=[-0.5, 1.1], row=1)

fig.add_trace(go.Scatter(x=x_cheb_err, y=err_cheb, name="err", mode='markers', marker=marker, legendgroup='2', showlegend=False), row=2, col=1)
fig.update_yaxes(type="log", exponentformat='e', row=2)

fig.add_trace(go.Scatter(x=x_cheb_err, y=cond_cheb, name="cond", mode='markers', marker=marker, legendgroup='3', showlegend=False), row=3, col=1)
fig.update_yaxes(type="log", exponentformat='e', row=3)

fig.add_trace(go.Scatter(x=x_cheb_err, y=dd_cheb, name="erreur rel / conditionnement", mode='markers', marker=marker, legendgroup='4', showlegend=False), row=4, col=1)
fig.update_yaxes(type="log", exponentformat='e', row=4, col=1)


buttons=[dict(label="cheb.", method="update", args=[{"x":[xplot, xplot, xplot, xk_cheb, x_cheb_err, x_cheb_err, x_cheb_err], 
                                                     "y":[f(xplot), pn_cheb, pn_cheb_ref, yk_cheb, err_cheb, cond_cheb, dd_cheb]}]),
         dict(label="equi.", method="update", args=[{"x":[xplot, xplot, xplot, xk_equi, x_equi_err, x_equi_err, x_equi_err], 
                                                     "y":[f(xplot), pn_equi, pn_equi_ref, yk_equi, err_equi, cond_equi, dd_equi]}])]

fig.update_layout(updatemenus=[dict(type="buttons", direction="right", buttons=buttons, x=0., xanchor="left", y=1.1, yanchor="top")], 
                  height=1500, legend_tracegroupgap = 320, legend_groupclick="toggleitem")

## Comparaison des méthodes

In [ ]:
n = 60
xmin = -1.
xmax = 1.

# point d'interpolation
xk_equi = np.linspace(xmin, xmax, n+1)
yk_equi = f(xk_equi)
xk_cheb = cheb_points(xmin, xmax, n+1)
yk_cheb = f(xk_cheb)

# points auxquels le polynome d'interpolation sera évalués pour le tracé
xplot = np.linspace(xmin, xmax, 500)

# polynome calculé par la méthode de Newton
pn_equi_new = newton_interp(xk_equi, yk_equi, xplot)
pn_equi_lag = improved_lag_interp(xk_equi, yk_equi, xplot)

pn_cheb_new = newton_interp(xk_cheb, yk_cheb, xplot)
pn_cheb_lag = improved_lag_interp(xk_cheb, yk_cheb, xplot)

# calcul de reference à précision fixée
mp.prec = 113
xk_equi_ref = mp.mpf('1') * xk_equi 
yk_equi_ref = f(xk_equi_ref)
xk_cheb_ref = mp.mpf('1') * xk_cheb 
yk_cheb_ref =  f(xk_cheb_ref)
xplot_ref = mp.mpf('1') * xplot 

pn_equi_ref_new = newton_interp(xk_equi_ref, yk_equi_ref, xplot_ref)
pn_equi_ref_new = np.array(pn_equi_ref_new, dtype=float)
pn_equi_ref_lag = improved_lag_interp(xk_equi_ref, yk_equi_ref, xplot_ref)
pn_equi_ref_lag = np.array(pn_equi_ref_lag, dtype=float)

pn_cheb_ref_new = newton_interp(xk_cheb_ref, yk_cheb_ref, xplot_ref)
pn_cheb_ref_new = np.array(pn_cheb_ref_new, dtype=float)
pn_cheb_ref_lag = improved_lag_interp(xk_cheb_ref, yk_cheb_ref, xplot_ref)
pn_cheb_ref_lag = np.array(pn_cheb_ref_lag, dtype=float)

# points auxquels le calcul de l'erreur et le conditionnement relatif local sont calculés
x_equi_err = np.linspace(xmin, xmax, 100)
# on retire les points xk de x_err
x_equi_err = np.setdiff1d(x_equi_err, xk_equi)
x_equi_err_ref = mp.mpf('1') * x_equi_err

p_equi_err_new = newton_interp(xk_equi, yk_equi, x_equi_err)
p_equi_err_ref_new = newton_interp(xk_equi_ref, yk_equi_ref, x_equi_err_ref)
err_equi_new = np.abs((p_equi_err_new - p_equi_err_ref_new)/p_equi_err_ref_new)
err_equi_new = np.array(err_equi_new, dtype=float)

p_equi_err_lag = improved_lag_interp(xk_equi, yk_equi, x_equi_err)
p_equi_err_ref_lag = improved_lag_interp(xk_equi_ref, yk_equi_ref, x_equi_err_ref)
err_equi_lag = np.abs((p_equi_err_lag - p_equi_err_ref_lag)/p_equi_err_ref_lag)
err_equi_lag = np.array(err_equi_lag, dtype=float)

x_cheb_err = np.linspace(xmin, xmax, 100)
# on retire les points xk de x_err
x_cheb_err = np.setdiff1d(x_cheb_err, xk_cheb)
x_cheb_err_ref = mp.mpf('1') * x_cheb_err

p_cheb_err_new = newton_interp(xk_cheb, yk_cheb, x_cheb_err)
p_cheb_err_ref_new = newton_interp(xk_cheb_ref, yk_cheb_ref, x_cheb_err_ref)
err_cheb_new = np.abs((p_cheb_err_new - p_cheb_err_ref_new)/p_cheb_err_ref_new)
err_cheb_new = np.array(err_cheb_new, dtype=float)

p_cheb_err_lag = improved_lag_interp(xk_cheb, yk_cheb, x_cheb_err)
p_cheb_err_ref_lag = improved_lag_interp(xk_cheb_ref, yk_cheb_ref, x_cheb_err_ref)
err_cheb_lag = np.abs((p_cheb_err_lag - p_cheb_err_ref_lag)/p_cheb_err_ref_lag)
err_cheb_lag = np.array(err_cheb_lag, dtype=float)
    
# conditionnement
cond_equi_new = compute_cond(xk_equi, yk_equi, np.array(p_equi_err_ref_new, dtype=float), x_equi_err)
cond_cheb_new = compute_cond(xk_cheb, yk_cheb, np.array(p_cheb_err_ref_new, dtype=float), x_cheb_err)
cond_equi_lag = compute_cond(xk_equi, yk_equi, np.array(p_equi_err_ref_lag, dtype=float), x_equi_err)
cond_cheb_lag = compute_cond(xk_cheb, yk_cheb, np.array(p_cheb_err_ref_lag, dtype=float), x_cheb_err)
    
# erreur / conditionnement 
dd_equi_new = err_equi_new/cond_equi_new
dd_cheb_new = err_cheb_new/cond_cheb_new
dd_equi_lag = err_equi_lag/cond_equi_lag
dd_cheb_lag = err_cheb_lag/cond_cheb_lag

fig = make_subplots(rows=4, cols=1, vertical_spacing=0.1,
                    subplot_titles=("Approximation polynomiale", "Erreur relative", 
                                    "Conditionnement relatif local", "Erreur relative / Conditionnement"))

fig.add_trace(go.Scatter(x=xplot, y=f(xplot), name="f(x)", legendgroup='1'), row=1, col=1)
fig.add_trace(go.Scatter(x=xplot, y=pn_cheb_new, name=f"pn(x) (Newton)", legendgroup='1'), row=1, col=1)
fig.add_trace(go.Scatter(x=xplot, y=pn_cheb_ref_new, name=f"pn(x) ref (Newton)", legendgroup='1'), row=1, col=1)
fig.add_trace(go.Scatter(x=xplot, y=pn_cheb_lag, name=f"pn(x) (Form. bary)", legendgroup='1'), row=1, col=1)
fig.add_trace(go.Scatter(x=xplot, y=pn_cheb_ref_lag, name=f"pn(x) ref (Form. bary)", legendgroup='1'), row=1, col=1)
fig.add_trace(go.Scatter(x=xk_equi, y=yk_equi, mode='markers', name="pts d'interpolation", legendgroup='1'), row=1, col=1)
fig.update_yaxes(range=[-0.5, 1.1], row=1)

fig.add_trace(go.Scatter(x=x_equi_err, y=err_equi_new, name="err (Newton)", mode='markers', marker=marker, legendgroup='2'), row=2, col=1)
fig.add_trace(go.Scatter(x=x_equi_err, y=err_equi_lag, name="err (Form. bary)", mode='markers', marker=marker2, legendgroup='2'), row=2, col=1)
fig.update_yaxes(type="log", exponentformat='e', row=2)

fig.add_trace(go.Scatter(x=x_equi_err, y=cond_equi_new, name="cond (Newton)", mode='markers', marker=marker, legendgroup='3'), row=3, col=1)
fig.add_trace(go.Scatter(x=x_equi_err, y=cond_equi_lag, name="cond (Form. bary)", mode='markers', marker=marker2, legendgroup='3'), row=3, col=1)
fig.update_yaxes(type="log", exponentformat='e', row=3)

fig.add_trace(go.Scatter(x=x_equi_err, y=dd_equi_new, name="erreur rel / conditionnement <br> (Newton)", mode='markers', marker=marker, legendgroup='4'), row=4, col=1)
fig.add_trace(go.Scatter(x=x_equi_err, y=dd_equi_lag, name="erreur rel / conditionnement <br> (Form. bary)", mode='markers', marker=marker2, legendgroup='4'), row=4, col=1)
fig.update_yaxes(type="log", exponentformat='e', row=4, col=1)

buttons=[dict(label="cheb.", method="update", args=[{"x":[xplot, xplot, xplot, xplot, xplot, xk_cheb, x_cheb_err, x_cheb_err, x_cheb_err, x_cheb_err, x_cheb_err, x_cheb_err], 
                                                     "y":[f(xplot), pn_cheb_new, pn_cheb_ref_new, pn_cheb_lag, pn_cheb_ref_lag, yk_cheb, err_cheb_new, err_cheb_lag, 
                                                          cond_cheb_new, cond_cheb_lag, dd_cheb_new, dd_cheb_lag]}]),
         dict(label="equi.", method="update", args=[{"x":[xplot, xplot, xplot, xplot, xplot, xk_equi, x_equi_err, x_equi_err, x_equi_err, x_equi_err, x_equi_err, x_equi_err], 
                                                     "y":[f(xplot), pn_equi_new, pn_equi_ref_new, pn_equi_lag, pn_equi_ref_lag, yk_equi, err_equi_new, err_equi_lag, 
                                                          cond_equi_new, cond_equi_lag, dd_equi_new, dd_equi_lag]}])]

fig.update_layout(updatemenus=[dict(type="buttons", direction="right", buttons=buttons, x=0., xanchor="left", y=1.1, yanchor="top")], 
                  height=1500, legend_tracegroupgap = 320, legend_groupclick="toggleitem")